In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from IPython.display import Image
from IPython.display import display, HTML
from IPython.display import clear_output

In [ ]:
def face_cards(card):
    #This function converts the values of the cards, including face cards, into the number they count for in blackjack
    if card.lower() in ['jack', 'queen', 'king']:
        return 10
    elif card.lower()=='ace':
        #The choice of 1 or 11 is subject to player and rules
        return int(input('Choose 1 or 11 '))
    else:
        return int(card)

In [ ]:
def blackjack():
    #Pull a deck
    response=requests.get('https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1')
    deck=response.json()
    deck_id=deck['deck_id']
    
    #Get two cards for player
    cards=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=2')
    
    #Display the images for these two cards
    url_list=[cards.json()['cards'][i]['image'] for i in range(len(cards.json()['cards']))]
    html_code = f'<img src="{url_list[0]}" style="float: left; margin-right: 10px;"><img src="{url_list[1]}" style="float: left;">'
    display(HTML(html_code))
    print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Dealer\'s card:')

    #Get one card for dealer and display
    dealer_card=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1')
    image_url = dealer_card.json()['cards'][0]['image']
    display(Image(url=image_url))
    
    #Isolate the values of our cards and the dealer's card
    dealer_value=[dealer_card.json()['cards'][0]['value']]
    values=[cards.json()['cards'][i]['value'] for i in range(len(cards.json()['cards']))]
    count=sum([face_cards(x) for x in values])

    status=0  #Status 0 means playing, Status 1 means standing, Status 2 means bust
    
    #Check that we are playing
    while status==0:
         #First, check if we are bust
        if count>21:
            status=2
            break
        
        #If we are not bust, let's see if we would like to hit or stand
        print(f'\nYour current count is {count}')
        time.sleep(1)
        action=input('\nHit or Stand? ')
        if action.lower()=='stand':
            status=1

        elif action.lower()=='hit':
            #If we hit, we should pull another card from the deck
            status=0
            cards=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1')
            
            #Display the new card
            image_url = cards.json()['cards'][0]['image']
            display(Image(url=image_url))

            #Get the number from the card and update the count
            values=cards.json()['cards'][0]['value']
            count+=face_cards(values)
    
    if status==2:
        #If we bust, the game is immediately over
        print("\nBUST! You Lost")

    elif status==1:
        #If we are in, it moves to the dealer
        print(f'\nYou stood on {count}')
        time.sleep(1)
        print('\nThe Dealer will stand on 17')
        dealer_count=face_cards(dealer_value[0])
        time.sleep(1)

        #Standard rule where dealer stands on 17
        while dealer_count<17:
            #New dealer card
            dealer_card=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1')

            #Display card
            image_url = dealer_card.json()['cards'][0]['image']
            display(Image(url=image_url))

            #Get the numerical value of the card and update the dealer count
            dealer_count+=face_cards(dealer_card.json()['cards'][0]['value'])
            print(f'\nDealer is currently on: {dealer_count}')
            time.sleep(1.5)

        if dealer_count>count and dealer_count<22:
            #Loses to dealer
            print(f'Dealer got {dealer_count}, you got {count}, you lose')
        elif dealer_count>21:
            #Dealer Bust
            print(f'Dealer bust on {dealer_count}')
        elif dealer_count==count:
            #Bet Returned
            print("Tie, bet returned")
        else:
            #Win
            print(f'Dealer got {dealer_count}, you got {count}, you win')

Easy way to play hands on repeat, if you would like to keep track of you previous hand, just delete the clear_output line

In [ ]:
try:
  while True:
    blackjack()
    time.sleep(2.5)
    clear_output(wait=True)
    time.sleep(1.5)
except KeyboardInterrupt:
    print('Ended')